## Creación de cluster sencillo que muestre la replicidad y la prueba de fallos de Apache Cassandra

Antes que nada, es importante instalar la herramienta llamada Docker. Docker es una plataforma de software que le permite crear, probar e implementar aplicaciones rápidamente. Docker empaqueta software en unidades estandarizadas llamadas contenedores que incluyen todo lo necesario para que el software se ejecute, incluidas bibliotecas, herramientas de sistema, código y tiempo de ejecución.  Su versatilidad y la forma en la que proporciona una manera estándar de ejecutar código lo convierte en un gran aliado para la base de datos NoSQL Apache Cassandra. Por lo tanto, antes de seguir el tutorial, es importante tener descargado la herramienta, puede instalarlo en el siguiente [link](https://docs.docker.com/desktop/windows/install/)

Se usará Docker para construir un clúster Cassandra con tres nodos

### 1. Tire de la imagen de Cassandra

In [ ]:
docker pull cassandra

###  2. Cree el primer nodo

In [ ]:
docker run --name cas1 -p 9042:9042 -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d cassandra

### 3. Cree el segundo nodo

In [ ]:
docker run --name cas2 -e CASSANDRA_SEEDS="$(docker inspect --format='{{ .NetworkSettings.IPAddress }}' cas1)" -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d cassandra

### 4. Cree el tercer nodo

In [ ]:
docker run --name cas3 -e CASSANDRA_SEEDS="$(docker inspect --format='{{ .NetworkSettings.IPAddress }}' cas1)" -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter2 -d cassandra

### 5. Verificar estado del cluster


In [ ]:
docker exec -it cas1 nodetool status

docker exec -it cas2 nodetool status

docker exec -it cas3 nodetool status

### 6. Conectarse a la utilidad de consola CQL (cqlsh)

In [ ]:
docker exec -it cas1 cqlsh

### 7. Se crea el espacio de llaves con un factor de replicación 3, esto con el objetivo de que las inserciones se realicen en los 3 nodos

In [ ]:
CREATE KEYSPACE patient WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 3};

### 8. Se realizan diversas inserciones a la tabla

In [ ]:
INSERT INTO patient.exam (patient_id, id, date, details) values (1,1,now(), 'primer examen del paciente 1');

INSERT INTO patient.exam (patient_id, id, date, details) values (1,2,now(), 'segundo examen del paciente 1');

INSERT INTO patient.exam (patient_id, id, date, details) values (2,1,now(), 'primer examen del paciente 2');

INSERT INTO patient.exam (patient_id, id, date, details) values (3,1,now(), 'primer examen del paciente 3');

### 9. Se realiza una consulta para verificar que las inserciones se realizaron correctamente

In [ ]:
select * from patient.exam where patient_id=1;

### 10. Se sale del nodo para realizar pruebas de replicación

In [ ]:
quit

### 11. Se ingresa nuevamente a la utilidad de la consola desde el nodo 1

In [ ]:
docker exec -it cas1 cqlsh

### 12. Se realiza nuevamente una inserción de datos 

In [ ]:
INSERT INTO patient.exam (patient_id, id, date, details) values (9,1,now(), 'primer examen del paciente 9');

### 13. Se verifica el contenido de la tabla

In [ ]:
select * from patient.exam where patient_id=9;

### 14. Se sale del nodo 1 y se procede a ingresar al nodo 3 para verificar la replicación de los datos

In [ ]:
quit

docker exec -it cas3 cqlsh

select * from patient.exam where patient_id=9;

### 15. Se detienen dos de los nodos de manera voluntaria para verificar su capacidad de replicación ante eventual fallos.

In [ ]:
docker stop cas2

docker stop cas3

### 16. Se ingresan nuevamente datos, para verificar que estos datos se replican aún con los nodos caídos

In [ ]:
INSERT INTO patient.exam (patient_id, id, date, details) values (10,1,now(), 'primer examen del paciente 10');


### 17. Se levantan nuevamente los nodos 

In [ ]:
docker start cas2

docker start cas3

### 18. Se ejecuta la utilidad de consola CQL desde la 3

In [ ]:
docker exec -it cas3 cqlsh

### 19. Se realiza una nueva consulta para ver los datos replicados en el nodo que se encontraba apagado.


In [ ]:
select * from patient.exam where patient_id=10;

Con esto, se termina el tutorial, para más información, verificar el video tutorial del siguiente [link](https://youtu.be/rStxffn9EcQ)